RayOpt Tutorial
========



In [1]:
%pylab inline
import warnings
import numpy as np
import matplotlib.pyplot as plt

import rayopt as ro

# ignore matplotlib and numpy warning each other
warnings.simplefilter("ignore", FutureWarning)
# ignore floating point exceptions
np.seterr(divide="ignore", invalid="ignore")
# by default only print 4 significant digits
np.set_printoptions(precision=4)

Populating the interactive namespace from numpy and matplotlib


In [2]:
s = ro.system_from_yaml("""
object:
    radius: 1
elements:
- {}
- {distance: 1, material: 1.5, roc: 5, radius: 1}
- {distance: .5, material: 1.0}
- {}
""")
s.update()
print(s)

c:\users\sujay\appdata\local\programs\python\python37-32\lib\site-packages\rayopt\formats.py:89: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dat = yaml.load(text)


TypeError: __init__() got an unexpected keyword argument 'radius'

In [ ]:
ro.Analysis(s)

In [ ]:
lib = ro.Library.one()
for g in lib.session.query(
    ro.library.Material).filter(
    ro.library.Material.name.contains("N-BK7")):
    print(g.name, g.catalog.name, g.catalog.source)

In [ ]:
s[1].material = lib.get("material", "SCHOTT-BK|N-BK7", source="rii")
s.object.angle = np.deg2rad(5)
s.fields = 0, .7, 1.
s.update()
s.paraxial.resize()
s.resize_convex()
s.paraxial.refocus()
ro.Analysis(s)

In [ ]:
s.object.pupil.radius = .1
q = ro.GaussianTrace(s)
fig, ax = plt.subplots()
s.plot(ax)
q.plot(ax, color="red", scale=10)
s.object.pupil.radius = 1.

In [ ]:
print(q)

In [ ]:
p = ro.PolyTrace(s, kmax=3)
print(p)

In [ ]:
s.paraxial.height[1], s.paraxial.lagrange, p.stvwof[-1, 0, 1]

In [ ]:
q = 1
#q *= 2
#q *= s.paraxial.height[1]
#q *= s.paraxial.lagrange
_ = p.seidel(*p.st())
print(np.array([v*q for k, v in _[:5]]))
print("\n".join(s.paraxial.print_transverse3()))

In [ ]:
print("\n".join(p.print_trace()))

In [ ]:
y = np.atleast_2d([1., 0, 0])
-s[1].surface_sag(y), p.stvwof[1, -1].view(p.Simplex)(*y.T[0]**2, [0.], [0.])